In [1]:
from sklearn.datasets import make_circles, make_moons, make_swiss_roll, make_s_curve
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from cvxopt import matrix
from cvxopt import solvers
from numpy import linalg as LA

In [2]:
def gaussian_kernel(x1, x2, sigma=0.1):
    return np.exp(-LA.norm(x1-x2, axis=-1)**2 / (2*sigma**2))

def poly_kernel(x1, x2, degree=3, r=2, gamma=1):
    return (np.dot(x1.T, x2) * gamma + r) ** degree

def laplacian_kernel(x1, x2, sigma=0.1):
    return np.exp(-LA.norm(x1-x2, axis=-1, ord=1) / sigma)

In [3]:
class svm:
    def __init__(self, C=10, kernel='rbf', sigma=0.1, degree=3, r=1, gamma=1):
        self.C = C
        self.kernel = kernel
        self.sigma = sigma
        self.degree = degree
        self.r = r
        self.gamma = gamma
    
    def _kernel_function(self, x1, x2):
        if self.kernel == "rbf":
            return gaussian_kernel(x1, x2, self.sigma)
        elif self.kernel == "poly":
            return poly_kernel(x1, x2, self.degree, self.r, self.gamma)
        elif self.kernel == "laplacian":
            return laplacian_kernel(x1, x2, self.sigma)
        else:
            print("undefined kernel")
            return None
        
    def _get_kernel_matrix(self, X_train, y_train):
        m,n = X_train.shape
        y_train = y_train.reshape(-1,1) * 1.
        K = np.zeros((m, m))
        for i, x_i in enumerate(X_train):
            for j, x_j in enumerate(X_train):
                K[i, j] = self._kernel_function(x_i, x_j)
        return K
    
    def fit(self, X_train, y_train):
        print("computing gram matrix")
        m,n = X_train.shape
        y_train = y_train.reshape(-1,1) * 1.
        K = self._get_kernel_matrix(X_train, y_train)
        K = np.outer(y_train, y_train) * K
        P = matrix(K)
        q = matrix(-np.ones(m))
        G = matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
        h = matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        A = matrix(y_train, (1, m))
        b = matrix(0.)
        #Run solver
        sol = solvers.qp(P, q, G, h, A, b)
        alphas = np.array(sol['x'])
        alphas = alphas.reshape(alphas.shape[0])
        sup_vec_idx = np.argwhere(alphas > 1e-4)
        sup_vec_idx = sup_vec_idx.reshape(sup_vec_idx.shape[0])
        X_sup_vec = X_train[sup_vec_idx]
        y_sup_vec = y_train[sup_vec_idx]
        alphas_sup_vec = alphas[sup_vec_idx]
        print("computing the bias term")
        y_sup_vec = y_sup_vec.reshape(y_sup_vec.shape[0])
        sup_vec_num = sup_vec_idx.shape[0]
        pred_mtx = np.zeros(sup_vec_num)
        b = 0
        for i in range(sup_vec_num):
            for j in range(sup_vec_num):
                pred_mtx[j] = self._kernel_function(X_sup_vec[j], X_sup_vec[0])
            b += y_sup_vec[i] - np.sum(alphas_sup_vec * y_sup_vec * pred_mtx)
        b /= sup_vec_num
        self.b = b
        self.sup_vec_idx = sup_vec_idx
        self.X_sup_vec = X_sup_vec
        self.y_sup_vec = y_sup_vec
        self.alphas_sup_vec = alphas_sup_vec
        
    def evaluate(self, X_test, y_test):
        preds = []
        sup_vec_num = self.sup_vec_idx.shape[0]
        pred_mtx = np.zeros(sup_vec_num)
        for Xt in X_test:
            for i in range(sup_vec_num):
                pred_mtx[i] = self._kernel_function(self.X_sup_vec[i], Xt)
            pred = np.sign(np.sum(self.alphas_sup_vec * self.y_sup_vec * pred_mtx))
            preds.append(int(pred))
        preds = np.array(preds)
        acc = np.sum((preds == y_test).astype("int")) / y_test.shape[0]
        return acc

In [4]:
X, y = make_moons(n_samples=2000, noise=0.2)
colors = ['red','blue']
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=matplotlib.colors.ListedColormap(colors), s=2)
y[y == 0] = -1
seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [9]:
clf = svm(kernel="poly", degree=10, r=3, gamma=1)
clf.fit(X_train, y_train)

computing gram matrix
     pcost       dcost       gap    pres   dres
 0: -9.1974e+02 -5.8620e+04  1e+05  4e-01  1e-06
 1: -8.1645e+02 -1.4288e+04  2e+04  5e-02  8e-07
 2: -8.2427e+02 -8.2532e+03  1e+04  2e-02  4e-07
 3: -8.4977e+02 -5.3576e+03  6e+03  9e-03  3e-07
 4: -9.2455e+02 -3.1076e+03  3e+03  4e-03  3e-07
 5: -9.6920e+02 -2.2676e+03  2e+03  2e-03  3e-07
 6: -9.8808e+02 -1.9353e+03  1e+03  1e-03  5e-07
 7: -9.9772e+02 -1.7161e+03  9e+02  6e-04  3e-07
 8: -1.0012e+03 -1.5991e+03  8e+02  4e-04  3e-07
 9: -1.0102e+03 -1.4473e+03  6e+02  2e-04  4e-07
10: -1.0101e+03 -1.4071e+03  5e+02  2e-04  4e-07
11: -1.0093e+03 -1.3381e+03  4e+02  1e-04  4e-07
12: -1.0056e+03 -1.2951e+03  4e+02  1e-04  3e-07
13: -1.0036e+03 -1.2202e+03  3e+02  5e-05  3e-07
14: -1.0052e+03 -1.1859e+03  2e+02  4e-05  2e-07
15: -1.0047e+03 -1.1691e+03  2e+02  3e-05  2e-07
16: -1.0105e+03 -1.1316e+03  2e+02  2e-05  2e-07
17: -1.0135e+03 -1.1059e+03  1e+02  1e-05  2e-07
18: -1.0134e+03 -1.0937e+03  1e+02  9e-06  2e-07

In [10]:
clf.evaluate(X_test, y_test)

0.875